In [1]:
import SimpleITK as sitk
import numpy as np

# File paths (anonymized)
ct_path = "/path/to/anonymized/CT.nrrd"
mask_path = "/path/to/anonymized/segmentations/GTV-1.nrrd"
study_id = "LUNG1-001"  # Example ID, adapt as needed

# Load the CT and mask
ct_image = sitk.ReadImage(ct_path)
mask_image = sitk.ReadImage(mask_path)

# Extract metadata
spacing = ct_image.GetSpacing()  # [x_spacing, y_spacing, z_spacing]
origin = ct_image.GetOrigin()    # [x_origin, y_origin, z_origin]

# Compute bounding box
label_filter = sitk.LabelShapeStatisticsImageFilter()
label_filter.Execute(mask_image)

bounding_box = label_filter.GetBoundingBox(1)  # [x_min, y_min, z_min, x_size, y_size, z_size]

# Extract voxel-level bounding box coordinates
x_min, y_min, z_min = bounding_box[0], bounding_box[1], bounding_box[2]
x_size, y_size, z_size = bounding_box[3], bounding_box[4], bounding_box[5]
x_max, y_max, z_max = x_min + x_size - 1, y_min + y_size - 1, z_min + z_size - 1

# Compute center coordinates in physical space
coordX = (x_min + x_max) / 2 * spacing[0] + origin[0]
coordY = (y_min + y_max) / 2 * spacing[1] + origin[1]
coordZ = (z_min + z_max) / 2 * spacing[2] + origin[2]

# Compute diameter (mean physical size along dimensions)
diameter_mm = np.mean([
    (x_max - x_min + 1) * spacing[0],  # Physical size in X
    (y_max - y_min + 1) * spacing[1],  # Physical size in Y
    (z_max - z_min + 1) * spacing[2]   # Physical size in Z
])

# Format the output for the model
output = f"{study_id}\t{coordX:.6f}\t{coordY:.6f}\t{coordZ:.6f}\t{diameter_mm:.6f}"
print("Model Input:\n", output)

Model Input:
LUNG1-001	80.566406	-196.351562	-456.500000	81.872396
